In [1]:
import sys
sys.path.insert(0, '../src/')
sys.path.insert(0, '../')

import django
django.setup()

/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [57]:
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
from apps.processing.ala.models import SamplingFeature, Observation
from apps.processing.ala.util import util

from importlib import import_module
from django.conf import settings
from apps.common.models import Process, Property

import pytz
import time
from psycopg2.extras import DateTimeTZRange
from django.utils.dateparse import parse_datetime
from dateutil.parser import parse
from datetime import datetime
from functools import partial

# from luminol.anomaly_detector import AnomalyDetector
import luminol.anomaly_detector as lad

from apps.utils.time import UTC_P0100
from apps.common.util.util import generate_intervals
# from apps.ad.anomaly_detection import *
import apps.ad.anomaly_detection as ad

feature = SamplingFeature.objects.get(id_by_provider='11359201')
observed_property = Property.objects.get(name_id='air_temperature')

from apps.mc.api.views import get_observations, parse_date_range, get_empty_slots

from apps.processing.ala.models import SamplingFeature, Observation
from apps.common.models import Process, Property, TimeSeries
from psycopg2.extras import DateTimeTZRange

In [76]:
topic = "drought"
prop = "air_temperature"
phenomenon_date_from = pd.to_datetime("2019-01-01")
phenomenon_date_to = pd.to_datetime("2019-01-30")
provider_model = Observation

def detect_anomalies(
    phenomenon_date_from = phenomenon_date_from,
    phenomenon_date_to = phenomenon_date_to,
    detector_method='bitmap_mod',
    use_baseline=True,
    shift=True,
    extend_range=True,
    detector_params={
        "precision": 6,
        "lag_window_size": 96,
        "future_window_size": 96,
        "chunk_size": 2
    },
    topic = topic,
    prop = prop,
    provider_model = provider_model,
):
    reload(ad)
    reload(lad)

    item = provider_model._meta.get_field('feature_of_interest').remote_field.model.objects.all()[5]

    observation_provider_name = f"{provider_model.__module__}.{provider_model.__name__}"
    topic_config = settings.APPLICATION_MC.TOPICS.get(topic)

    properties = topic_config['properties']
    ts_config = topic_config['time_series']
    prop_config = properties[prop]

    observation_provider_name = f"{Observation.__module__}.{Observation.__name__}"
    process_name_id = prop_config['observation_providers'][observation_provider_name]["process"]
    process = Process.objects.get(name_id=process_name_id)

    prop_item = Property.objects.get(name_id=prop)

    pt_range, day_from, day_to = parse_date_range(str(phenomenon_date_from), str(phenomenon_date_to))

    pt_range_z = DateTimeTZRange(
        pt_range.lower.replace(tzinfo=UTC_P0100),
        pt_range.upper.replace(tzinfo=UTC_P0100)
    )

    zero = parse_datetime(ts_config['zero'])
    frequency = ts_config['frequency']
    range_from = ts_config['range_from']
    range_to = ts_config['range_to']

    t = TimeSeries(
        zero=zero,
        frequency=frequency,
        range_from=range_from,
        range_to=range_to
    )
    t.full_clean()
    t.clean()

    feature_time_slots = get_empty_slots(t, pt_range_z)

    get_observations_func = partial(
        get_observations,
        feature_time_slots,
        prop_item,
        provider_model,
        item,
        process,
        t)

    return ad.get_timeseries(
    #     phenomenon_time_range=data_range,
        phenomenon_time_range=pt_range_z,
        num_time_slots=len(feature_time_slots),
        get_observations=get_observations_func,
        detector_method=detector_method,
        detector_params=detector_params,
        shift=shift,
        use_baseline=use_baseline,
        extend_range=extend_range,
    )

In [58]:
colors = ['r', 'g', 'c', 'm', 'y', 'k']

def plot(detectors):
    fig, ax1 = plt.subplots(figsize=(20,7))
    first_result = detectors[list(detectors.keys())[0]]
    
    ts = pd.DataFrame({
        'values': [float(n) for n in first_result["property_values"]]
    })
    
    values_line = ts['values'].plot.line(ax=ax1, color='b')
    ax1.set_ylabel('values', color='b')
    ax1.tick_params('y', colors='b')
    
    lns = [values_line.get_lines()[0]]
    
    for i in range(len(detectors.keys())):
        detector = list(detectors.keys())[i]
        color = colors[i]
        anomalies = detectors[detector]["property_anomaly_rates"]

        ts = pd.DataFrame({
            'anomalies': anomalies
        })
        
        ax2 = ax1.twinx()
        anomalies_line = ts['anomalies'].plot.line(ax=ax2, color=color, label=detector)
        lns.append(anomalies_line.get_lines()[0])
#         if i == 1:
#             ax2.set_ylabel('anomalies', color='k')
        ax2.tick_params('y', colors=color)

    labs = [ln.get_label() for ln in lns]
    ax1.legend(lns, labs, loc=1)
    
    # baseName = f"{baserange.lower.date()}..{baserange.upper.date()}"
    rangeName = f"{first_result['phenomenon_time_range'].lower.date()}..{first_result['phenomenon_time_range'].upper.date()}"
    # plt.savefig(f"graphs/{baseName}_{rangeName}_window-{str(window_size)}_prec-{str(detector_params['precision'])}.png", format="png")
    # plt.savefig(f"graphs/{rangeName}_window-{str(window_size)}_prec-{str(detector_params['precision'])}.png", format="png")

In [66]:
def plot_anomalies(start_date, end_date):
#     start_date = start_date.date()
#     end_date = end_date.date()
    anomalies = {
        'Bitmap mod': detect_anomalies(start_date, end_date, "bitmap_mod", shift=False),
        'Bitmap mod shift': detect_anomalies(start_date, end_date, "bitmap_mod_shift"),
        'LinkedIn bitmap': detect_anomalies(start_date, end_date, "bitmap_detector"),
        'Default': detect_anomalies(start_date, end_date, "default_detector", detector_params={}, use_baseline=False, extend_range=False, shift=False),
        'Derivative': detect_anomalies(start_date, end_date, "derivative_detector", detector_params={}, use_baseline=False, extend_range=False, shift=False),
        'Exponential average': detect_anomalies(start_date, end_date, "exp_avg_detector", detector_params={}, use_baseline=False, extend_range=False, shift=False),
    #     'Absolute threshold': detect_anomalies(t_from, t_to, "absolute_threshold", detector_params={}, use_baseline=False, extend_range=False, shift=False),
    #     'Diff Percent': detect_anomalies(t_from, t_to, "diff_percent_threshold", detector_params={}, use_baseline=False, extend_range=False, shift=False),
    #     'Sign test': detect_anomalies(t_from, t_to, "sign_test", detector_params={}, use_baseline=False, extend_range=False, shift=False),
    }

    plot(anomalies)

In [77]:
t_from = "2019-01-01"
t_to = "2019-01-30"

interact(plot_anomalies,
     start_date=widgets.DatePicker(value=pd.to_datetime(t_from).date()),
     end_date=widgets.DatePicker(value=pd.to_datetime(t_to).date()))

interactive(children=(DatePicker(value=datetime.date(2019, 1, 1), description='start_date'), DatePicker(value=…

<function __main__.plot_anomalies(start_date, end_date)>